# 🔍 에프엠코리아 크롤러 (Selenium - 코랩 전용)
## 페이지네이션 문제 해결 버전

---
## 1️⃣ 환경 설정 (필수! 처음 1회)

In [ ]:
# 코랩용 Selenium 설치
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

print("✅ 설치 완료!")

---
## 2️⃣ 크롤러 클래스

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from urllib.parse import quote, urljoin
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import re


class FMKoreaCrawlerSelenium:
    """에프엠코리아 크롤러 (Selenium - 코랩 전용)"""

    def __init__(self, delay_min=3, delay_max=6):
        self.base_url = "https://www.fmkorea.com"
        self.delay_min = delay_min
        self.delay_max = delay_max
        self.driver = None
        
    def _init_driver(self):
        """코랩용 Chrome 드라이버 초기화"""
        print("🔄 브라우저 초기화 중...")
        
        chrome_options = Options()
        
        # 코랩 필수 옵션들
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--disable-software-rasterizer')
        chrome_options.add_argument('--disable-extensions')
        chrome_options.add_argument('--remote-debugging-port=9222')
        chrome_options.add_argument('--window-size=1920,1080')
        
        # 봇 탐지 우회
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        
        # User-Agent
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        # 드라이버 생성
        self.driver = webdriver.Chrome(options=chrome_options)
        
        # webdriver 탐지 우회
        self.driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
        })
        
        print("✅ 브라우저 준비 완료!")

    def _delay(self, extra=0):
        time.sleep(random.uniform(self.delay_min + extra, self.delay_max + extra))

    def _parse_date(self, date_str):
        if not date_str:
            return None
        try:
            today = datetime.now()
            date_str = date_str.strip()
            
            match = re.search(r'(\d{4})[.\-/](\d{1,2})[.\-/](\d{1,2})', date_str)
            if match:
                return datetime(int(match.group(1)), int(match.group(2)), int(match.group(3)))
            
            match = re.search(r'^(\d{1,2})[.\-/](\d{1,2})$', date_str)
            if match:
                return datetime(today.year, int(match.group(1)), int(match.group(2)))
            
            if '방금' in date_str or '초 전' in date_str:
                return today
            elif '분 전' in date_str:
                m = re.search(r'(\d+)', date_str)
                return today - timedelta(minutes=int(m.group(1))) if m else today
            elif '시간 전' in date_str:
                m = re.search(r'(\d+)', date_str)
                return today - timedelta(hours=int(m.group(1))) if m else today
            elif '일 전' in date_str:
                m = re.search(r'(\d+)', date_str)
                return today - timedelta(days=int(m.group(1))) if m else today
            return None
        except:
            return None

    def _scroll_down(self):
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    def search_posts(self, keyword, max_pages=50, max_posts=200, start_date=None, end_date=None):
        if self.driver is None:
            self._init_driver()
        
        posts = []
        visited_urls = set()
        
        if isinstance(start_date, str):
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
        if isinstance(end_date, str):
            end_date = datetime.strptime(end_date, '%Y-%m-%d')
        if start_date is None:
            start_date = datetime(2024, 1, 1)
        if end_date is None:
            end_date = datetime(2025, 12, 31)

        print(f"\n{'='*60}")
        print(f"🔍 '{keyword}' 검색 시작")
        print(f"📅 기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")
        print(f"{'='*60}\n")

        for page_num in range(1, max_pages + 1):
            search_url = f"{self.base_url}/search.php?search_keyword={quote(keyword)}&search_target=title_content&page={page_num}"
            
            print(f"📄 페이지 {page_num}/{max_pages}...")
            
            try:
                self.driver.get(search_url)
                self._delay()
                
                WebDriverWait(self.driver, 15).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )
                
                self._scroll_down()
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                
            except Exception as e:
                print(f"    ❌ 페이지 로드 실패: {e}")
                continue
            
            found_in_page = 0
            
            # 게시물 링크 찾기
            links_found = soup.find_all('a', href=re.compile(r'/\d+$'))
            
            for link in links_found:
                if len(posts) >= max_posts:
                    break
                
                href = link.get('href', '')
                title = link.get_text(strip=True)
                
                if not href or not title or len(title) < 3:
                    continue
                
                if not re.search(r'/\d+$', href):
                    continue
                
                full_url = urljoin(self.base_url, href)
                
                if full_url in visited_urls:
                    continue
                
                visited_urls.add(full_url)
                
                parent = link.find_parent(['li', 'tr', 'div'])
                date_str = ''
                date_parsed = None
                
                if parent:
                    date_elem = parent.find(['span', 'td'], class_=re.compile(r'date|time|regdate|side'))
                    if date_elem:
                        date_str = date_elem.get_text(strip=True)
                        date_parsed = self._parse_date(date_str)
                
                if date_parsed and not (start_date <= date_parsed <= end_date):
                    continue
                
                posts.append({
                    'title': title,
                    'url': full_url,
                    'keyword': keyword,
                    'date': date_str,
                    'date_parsed': date_parsed
                })
                found_in_page += 1
            
            print(f"    ✅ {found_in_page}개 발견 (총 {len(posts)}개)")
            
            if len(posts) >= max_posts:
                print(f"\n🎯 목표 달성!")
                break
            
            if found_in_page == 0 and page_num > 5:
                print(f"    ⚠️ 빈 페이지 연속. 중단.")
                break

        print(f"\n✅ 검색 완료! 총 {len(posts)}개")
        return posts

    def get_post_detail(self, url):
        if self.driver is None:
            self._init_driver()
        
        try:
            self.driver.get(url)
            self._delay()
            
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            
            for _ in range(3):
                self._scroll_down()
            
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
        except Exception as e:
            print(f"    ❌ 로드 실패: {e}")
            return None

        result = {'url': url, 'title': '', 'author': '', 'date': '', 'content': '', 'comments': []}

        # 제목
        elem = soup.select_one('span.np_18px_span') or soup.select_one('h1.title') or soup.find('title')
        if elem:
            result['title'] = elem.get_text(strip=True)

        # 작성자
        elem = soup.select_one('a.member') or soup.select_one('span.author')
        if elem:
            result['author'] = elem.get_text(strip=True)

        # 날짜
        elem = soup.select_one('span.date') or soup.select_one('span.time')
        if elem:
            result['date'] = elem.get_text(strip=True)

        # 본문
        elem = soup.select_one('div.xe_content') or soup.select_one('article')
        if elem:
            for tag in elem.find_all(['script', 'style', 'img', 'video', 'iframe']):
                tag.decompose()
            result['content'] = elem.get_text(separator='\n', strip=True)

        # 댓글
        for item in soup.select('div.fdb_lst_ul li, ul.fdb_lst_ul > li')[:50]:
            content_elem = item.select_one('div.xe_content')
            if content_elem:
                content = content_elem.get_text(strip=True)
                if content and len(content) > 1:
                    author_elem = item.select_one('a.member')
                    result['comments'].append({
                        'author': author_elem.get_text(strip=True) if author_elem else '',
                        'content': content
                    })

        return result

    def crawl_with_details(self, keyword, max_posts=50, start_date=None, end_date=None):
        posts = self.search_posts(keyword=keyword, max_posts=max_posts, start_date=start_date, end_date=end_date)
        
        if not posts:
            print("❌ 검색 결과 없음")
            return [], []
        
        posts_data = []
        comments_data = []
        
        print(f"\n📝 상세 크롤링 ({len(posts)}개)")
        print("-" * 60)
        
        for i, post in enumerate(posts, 1):
            print(f"[{i}/{len(posts)}] {post['title'][:40]}...")
            
            detail = self.get_post_detail(post['url'])
            if detail:
                posts_data.append({
                    'title': detail['title'] or post['title'],
                    'url': post['url'],
                    'author': detail['author'],
                    'date': detail['date'],
                    'content': detail['content'],
                    'keyword': keyword,
                    'comment_count': len(detail['comments'])
                })
                
                for comment in detail['comments']:
                    comments_data.append({
                        'post_title': detail['title'] or post['title'],
                        'post_url': post['url'],
                        'comment_author': comment['author'],
                        'comment_content': comment['content'],
                        'keyword': keyword
                    })
                
                print(f"    ✅ 댓글 {len(detail['comments'])}개")
            else:
                print(f"    ❌ 실패")
        
        print(f"\n{'='*60}")
        print(f"✅ 완료! 게시물: {len(posts_data)}개, 댓글: {len(comments_data)}개")
        
        return posts_data, comments_data

    def close(self):
        if self.driver:
            self.driver.quit()
            self.driver = None
            print("🔒 브라우저 종료")


print("✅ 크롤러 클래스 로드 완료!")

---
## 3️⃣ 크롤링 실행

In [ ]:
# 크롤러 초기화
crawler = FMKoreaCrawlerSelenium(delay_min=3, delay_max=6)

# 🔹 설정
KEYWORD = "마약"
MAX_POSTS = 50
START_DATE = "2024-01-01"
END_DATE = "2025-12-31"

# 크롤링 실행
posts_data, comments_data = crawler.crawl_with_details(
    keyword=KEYWORD,
    max_posts=MAX_POSTS,
    start_date=START_DATE,
    end_date=END_DATE
)

---
## 4️⃣ 결과 확인 및 저장

In [ ]:
# DataFrame 변환
posts_df = pd.DataFrame(posts_data)
comments_df = pd.DataFrame(comments_data)

print(f"📊 게시물: {len(posts_df)}개")
print(f"📊 댓글: {len(comments_df)}개")

if not posts_df.empty:
    print("\n📝 게시물 샘플:")
    display(posts_df[['title', 'date', 'comment_count']].head(10))

In [ ]:
# 저장 및 다운로드
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
posts_file = f"{KEYWORD}_게시물_{timestamp}.xlsx"
comments_file = f"{KEYWORD}_댓글_{timestamp}.xlsx"

posts_df.to_excel(posts_file, index=False)
comments_df.to_excel(comments_file, index=False)

print(f"✅ 저장 완료!")

from google.colab import files
files.download(posts_file)
files.download(comments_file)

In [ ]:
# 브라우저 종료
crawler.close()